# 5. 머신러닝 프로세스

데이터셋 분할과 모델 검증

## 1. x, y 나누기

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('./../../data/bigboongi/Fvote.csv', encoding='utf-8')
data.head()

,gender_female,gender_male,region_Chungcheung,region_Honam,region_Others,region_Sudo,region_Youngnam,edu,income,age,score_gov,score_progress,score_intention,vote,parties
0,0,1,0,0,0,0,1,3,3,3,2,2,4.0,1,2
1,0,1,0,0,1,0,0,2,3,3,2,4,3.0,0,3
2,0,1,0,1,0,0,0,1,2,4,1,3,2.8,1,4
3,1,0,0,0,0,1,0,2,1,3,5,4,2.6,1,1
4,0,1,0,0,0,1,0,1,2,4,4,3,2.4,1,1


In [4]:
X = data[data.columns[0:13]]  # X = data.loc[:, 'gender_female':'score_intention']
y = data[['vote']]

## 2. train-test 데이터셋 나누기

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [6]:
print(X_train.shape)
print(X_test.shape)

(158, 13)
(53, 13)


## 3. 모델 적용

In [7]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

### 가. 랜덤 없는 교차검증 cross_val_score

train data를 모두 한 번에 사용하지 않고 k개의 그룹으로 나누어 이 중 한 그룹을 빼고 k-1개의 그룹만 훈련을 함. 이걸 k번 반복.

In [8]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=5)
print('5개 테스트셋 정확도:', scores)
print('정확도 평균:', scores.mean())

5개 테스트셋 정확도: [0.71875    0.6875     0.8125     0.58064516 0.80645161]
정확도 평균: 0.7211693548387096


### 나. 랜덤 있는 교차검증 K-Fold

cross_val은 데이터셋의 순서가 있는 그대로 나눔
K-Fold는 훈련데이터에서 그룹을 나눌 때부터 랜덤(무작위 추출)하듯 섞는(shuffle) 방법을 이용 -> 표본이 섞여서 뽑힘. 편향성이 적음.

In [10]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X_train, y_train, cv=kfold)
print('5개 폴드의 정확도: ', scores)
print('정확도 평균: ', scores.mean())

5개 폴드의 정확도:  [0.71875    0.6875     0.625      0.70967742 0.77419355]
정확도 평균:  0.7030241935483872


### 3. 임의분할 교차검증

훈련데이터와 테스트 데이터를 구성할 때 다른 교차 검증에 사용되었던 데이터도 랜덤으로 선택되게 하는 방법
전체 데이터 중 일부는 훈련데이터 또는 테스트데이터 어디에서 선택되지 않을 수도 있음(앞서 cross_val, KFold는 한 번이라도 훈련에 사용됨)

In [11]:
from sklearn.model_selection import ShuffleSplit

shuffle_split = ShuffleSplit(test_size=0.5, train_size=0.5, random_state=42)
scores = cross_val_score(model, X_train, y_train, cv=shuffle_split)
print('교차검증 정확도: ', scores)
print('정확도 평균: ', scores.mean())

교차검증 정확도:  [0.73417722 0.69620253 0.70886076 0.73417722 0.65822785 0.67088608
 0.72151899 0.65822785 0.69620253 0.70886076]
정확도 평균:  0.6987341772151898


### 4. train-validity-test 분할과 교차검증

valid data도 나누어봄.
우선 train과 test로 나누고, train 데이터를 다시 train과 valid로 나눔. 7:3

In [12]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=1)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_val, y_train_val, random_state=2)

In [13]:
# 모델 훈련
model.fit(X_train, y_train)
scores = cross_val_score(model, X_train, y_train, cv=5)
print('교차검증 정확도: ', scores)
print('정확도 평균: ', scores.mean())

교차검증 정확도:  [0.58333333 0.66666667 0.70833333 0.65217391 0.65217391]
정확도 평균:  0.652536231884058


In [14]:
model.score(X_valid, y_valid)

0.65

In [15]:
model.score(X_test, y_test)

0.6981132075471698